In [1]:
import patient as p
import tensorflow as tf
import numpy as np
import myLib
import consts as c



In [2]:
c.WINDOW_SIZE = c.PREPREDICTION_LENGTH + 1000

tabSegments = []
for pStr in c.trainPatients:
    patient = p.patient(pStr)
    pos = patient.getPositiveSegments()
    neg = patient.getNegativesN(len(pos))
    tabSegments = tabSegments + pos + neg # add positves
    #tabSegments = tabSegments + patient.getNegativesN(len(tabSegments[-1][0]))
    


x_train,y_train = myLib.processDF(tabSegments)

tabSegments = []
patient = p.patient(c.validationPatient)
pos = patient.getPositiveSegments()
neg = patient.getNegativesN(len(pos))
tabSegments = tabSegments + pos + neg # add positves

x_validation,y_validation = myLib.processDF(tabSegments)

tabSegments = []
patient = p.patient(c.testPatient)
pos = patient.getPositiveSegments()
neg = patient.getNegativesN(len(pos))
tabSegments = tabSegments + pos + neg # add positves
x_test,y_test = myLib.processDF(tabSegments)




In [3]:
print(f"train data: {len(x_train)}\t validation: {len(x_validation)}\t test:{len(x_test)}\n")
unique, counts = np.unique(y_train, return_counts=True)

print(f"TRAIN:     Negatives: {counts[0]} Positives: {counts[1]}")
pos = counts[1]
neg = counts[0]
ratio = round(100 / len(x_train) * pos)
unique, counts = np.unique(y_validation, return_counts=True)
print(f"VALIDATION Negatives: {counts[0]} Positves: {counts[1]}")
unique, counts = np.unique(y_test, return_counts=True)
print(f"TEST       Negatives: {counts[0]} Positves: {counts[1]}")


print(f"\n{ratio} % of positives")




train data: 268	 validation: 24	 test:84

TRAIN:     Negatives: 134 Positives: 134
VALIDATION Negatives: 12 Positves: 12
TEST       Negatives: 42 Positves: 42

50 % of positives


In [7]:

def getModel1():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(1,activation="sigmoid"))

    #model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

    model.compile(
          optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
          loss=tf.keras.losses.BinaryCrossentropy(),
          metrics=[
              tf.keras.metrics.BinaryAccuracy(name='accuracy')
          ]
    )
    return model

METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

def getModel2(metrics=METRICS, output_bias=None):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(
          16, activation='relu',
          input_shape=(x_train.shape[-1],)),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.Dense(1, activation='sigmoid',
                         bias_initializer=output_bias),
  ])

  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
      loss=tf.keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model



In [8]:
model = getModel1()

model.fit(
    x_train,
    y_train,
    epochs=30,
    validation_data=(x_validation, y_validation), 
    verbose=1)

Epoch 1/30
9/9 [==============================] - 1s 33ms/step - loss: 0.6789 - accuracy: 0.7313 - val_loss: 2.1359 - val_accuracy: 0.3750
Epoch 2/30
9/9 [==============================] - 0s 17ms/step - loss: 0.5030 - accuracy: 0.8172 - val_loss: 2.8579 - val_accuracy: 0.3750
Epoch 3/30
9/9 [==============================] - 0s 19ms/step - loss: 0.4325 - accuracy: 0.8843 - val_loss: 1.0818 - val_accuracy: 0.4583
Epoch 4/30
9/9 [==============================] - 0s 17ms/step - loss: 0.3210 - accuracy: 0.8918 - val_loss: 1.5830 - val_accuracy: 0.3333
Epoch 5/30
9/9 [==============================] - 0s 17ms/step - loss: 0.2961 - accuracy: 0.8881 - val_loss: 2.1548 - val_accuracy: 0.2917
Epoch 6/30
9/9 [==============================] - 0s 17ms/step - loss: 0.4743 - accuracy: 0.8881 - val_loss: 1.3981 - val_accuracy: 0.4583
Epoch 7/30
9/9 [==============================] - 0s 17ms/step - loss: 0.3063 - accuracy: 0.9067 - val_loss: 1.8898 - val_accuracy: 0.4167
Epoch 8/30
9/9 [===========

In [9]:
val_loss = model.evaluate(x_test,y_test)
print(f"\nloss: {val_loss}")

predictions = model.predict(x_test)
predictions = [round(x[0]) for x in predictions]

for i in range(len(y_test)):
    print(f"{y_test[i]} => {predictions[i]}")

3/3 [==============================] - 0s 9ms/step - loss: 2.0814 - accuracy: 0.6071

loss: [2.081373691558838, 0.6071428656578064]
0.0 => 0
1.0 => 1
0.0 => 0
0.0 => 0
1.0 => 1
0.0 => 0
1.0 => 0
1.0 => 0
1.0 => 1
1.0 => 0
1.0 => 1
1.0 => 0
0.0 => 1
0.0 => 1
1.0 => 0
1.0 => 1
0.0 => 0
0.0 => 0
0.0 => 0
1.0 => 0
1.0 => 0
0.0 => 1
0.0 => 1
1.0 => 1
1.0 => 0
0.0 => 1
0.0 => 0
1.0 => 1
1.0 => 0
1.0 => 1
1.0 => 0
1.0 => 1
0.0 => 0
1.0 => 0
0.0 => 0
0.0 => 0
0.0 => 0
0.0 => 0
1.0 => 0
0.0 => 0
1.0 => 1
0.0 => 0
0.0 => 0
1.0 => 1
0.0 => 1
0.0 => 1
0.0 => 1
0.0 => 0
1.0 => 1
0.0 => 0
1.0 => 1
1.0 => 1
0.0 => 1
1.0 => 1
0.0 => 0
1.0 => 0
1.0 => 0
1.0 => 1
1.0 => 0
1.0 => 1
1.0 => 0
0.0 => 1
0.0 => 0
0.0 => 1
1.0 => 0
1.0 => 0
0.0 => 0
0.0 => 1
1.0 => 0
0.0 => 0
1.0 => 1
1.0 => 0
1.0 => 1
0.0 => 0
1.0 => 1
1.0 => 0
0.0 => 0
0.0 => 0
0.0 => 0
0.0 => 0
0.0 => 0
0.0 => 0
1.0 => 1
0.0 => 0
